In [1]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore, SPARQLStore
from rdflib import Graph, Namespace, URIRef

from xml.dom.minidom import parse
import xml.etree.ElementTree as etree
from Marc_to_Bibframe.Marc.marcWork import MarcWork
from Marc_to_Bibframe.Marc.marcInstance import MarcInstance
from Marc_to_Bibframe.Marc.marcItems import MarcItems
from Marc_to_Bibframe.Work.work import Work
from Marc_to_Bibframe.Instance.instance import Instance

import pysolr
from Solr.solr import create_doc
from Jena_cli.jena import UpadateJena

DELETE ALL GRAPHS

In [2]:
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/bibframe/update')
query_endpoint = 'http://localhost:3030/bibframe/query'
update_endpoint = 'http://localhost:3030/bibframe/update'
store.open((query_endpoint, update_endpoint))

In [5]:
#Delete all Named Graph

d = """DELETE { graph ?g { ?s ?p ?o } } 

WHERE {

graph ?g {?s ?p ?o.}

}"""

store.update(d)

SOLR

In [10]:
#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":6,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-1"}},\n  "status":"OK"}\n'

In [2]:
def UpadateSolr(path_marc, count, shelf):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(count)
        print(workMarc.Title().get('title'))

        #SOLR
        doc = create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)
        
        count += 1
    return docs, count

In [ ]:
count = 378
e = '3'
p = '5'
out = f'E{e}/P{p}'
path_marc = f'Input/{out}/koha.xml'

docs, _ = UpadateSolr(path_marc, count, f"E{e}.P{p}")

In [49]:
#solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.add(docs, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":348}}\n'

IMPORT LOTE

In [ ]:
count = 1
solr_docs = list()
for e in range(1, 3):
    for p in range(1,6):
        path_marc = f'Input/E{e}/P{p}/koha.xml'
        docs, count  = UpadateSolr(path_marc, count, f"E{e}.P{p}")
        #solr_docs.append(docs)
        solr_docs = solr_docs + docs
        #count += 1
        #print(count)

In [35]:
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.add(solr_docs, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":1736}}\n'

JENA

In [ ]:
count = 1
for i in range(1,3):
    e = i
    for p in range(1,6):
        out = f'E{e}/P{p}'
        path_marc = f'Input/{out}/koha.xml'
        shelf = f"E{e}.P{p}"
        count = UpadateJena(path_marc, count, shelf, out)


In [16]:
e = 2
count = 106
for p in range(1,6):
    out = f'E{e}/P{p}'
    path_marc = f'Input/{out}/koha.xml'
    count = UpadateJena(path_marc, count, shelf, out)

106
An introduction to RPG-RPG II programming
107
Programacion con el lenguaje Cobol
108
Dominando o PostgreSQL
109
Study notes in system dynamics.
110
TURBO PASCAL
111
FORTH, guia do usuário
112
Princípios em projetos de programas
113
Lenguajes de diagramas de flujo
114
Linguagem PASCAL
115
A simplified guide to FORTRAN programming
116
Lingugem C
117
Biblioteca do programador Clipper 5.01
118
Delphi 2
119
Manipulando bancos de dados com Delphi 2
120
SQL para Delphi 2
121
Cobol y sus aplicaciones en los negocios
122
Clipper
123
Pascal e técnicas de programação
124
I simpósio brasileiro de linguagens de programação
125
MUMPS: uma nova abordagem
126
turbo Pascal
127
Supercalc
128
Fortran
129
Fortran - Monitor
130
Quattro Pro 2
131
Desktop publishing com pagemaker IBM PC AT, PS.2 e compatíveis
132
Macro processors and techniques for portable software
133
TURBO BASIC
134
Princípios de sistemas de gerência de bancos de dados distribuídos
135
Sistemas de gerência de bancos de dados distribuí

http://id.loc.gov/vocabulary/languages/0 p does not look like a valid URI, trying to serialize this will break.


141
Dbase III plus


Exception: "http://id.loc.gov/vocabulary/languages/0 p" does not look like a valid URI, I cannot serialize this as N3/Turtle. Perhaps you wanted to urlencode it?

In [15]:
count

106